In [1]:
import re
import pandas as pd

'''
basedir = '/home/yannick/workspace/eclipse/workspace/jfreechart-1.0.10-rf'
sourcedir = 'tests'
fout = 'output/jfreechart-1.0.10-deckard-selected.xls'
fileName = 'data/post_cluster_vdb_50_0_allg_0.95_50'
annotation_line = 0
skip_method_name = ['suite', 'testEquals']
'''

'''
basedir = '/home/yannick/workspace/eclipse/workspace/gson-gson-parent-2.8.5/gson'
sourcedir = 'src/test/java'
fileName = 'data/gson_cluster_vdb_50_0_allg_0.95_50'
fout = 'output/gson-2.8.5-deckard-selected.xls'
annotation_line = 0
skip_method_name = []
'''

'''
basedir = '/home/yannick/workspace/eclipse/workspace/commons-lang-3.7'
sourcedir = 'src/test/java'
fileName = 'data/commons_lang_cluster_vdb_50_0_allg_0.95_50'
fout = 'output/commons-lang-3.7-deckard-selected.xls'
annotation_line = 1
skip_method_name = []
'''

'''
basedir = '/home/yannick/workspace/eclipse/workspace/commons-io-2.5'
sourcedir = 'src/test/java'
fileName = 'data/commons_io_cluster_vdb_50_0_allg_0.95_50'
fout = 'output/commons-io-2.5-deckard-selected.xls'
annotation_line = 1
skip_method_name = []
'''


basedir = '/home/yannick/workspace/eclipse/workspace/joda-time-2.10'
sourcedir = 'src/test/java'
fileName = 'data/jodatime_cluster_vdb_50_0_allg_0.95_50'
fout = 'output/joda-time-2.10-deckard-selected.xls'
annotation_line = 0
skip_method_name = []


method_name_pattern = re.compile(r'^(.*)void[\s]+(.*?)[\s]*\([\s]*\)')
path_pattern = re.compile(basedir + '/' + sourcedir + '/(.*)/(.*?).java')

columnNames = ['Clone Group ID', 'Source Folder', 'Package', 'Class', 'Method', 'Method Signature', 
               'Start Line', 'End Line', 'Start Offset', 'End Offset', '#PDG Nodes', '#Statements', 
               'Line Coverage(%)', 'Clone Group Size', 'Clone Group Info', 'Connected', 'Clone Pair Location', 
               '#Refactorable Pairs', 'Details']
df = pd.DataFrame(columns=columnNames)

def getMethodName(path, startLine, kind):
    if kind == 113:
        javaFile = open(path, 'r')
        line = javaFile.readlines()[int(startLine) - 1 + annotation_line]
        javaFile.close()
        m = method_name_pattern.match(line)
        if m != None:
            return m.group(2)

def getCUInfo(path):
    m = path_pattern.match(path)
    return sourcedir, m.group(1).replace('/', '.'), m.group(2)

def checkPair(pair):
    pair1 = pair[0]
    pair2 = pair[1]
    return pair1[1] == sourcedir and pair2[1] == sourcedir and pair1[2] == pair2[2] \
        and pair1[4] not in skip_method_name and pair2[4] not in skip_method_name

def addRecords(records):
    for i, record in enumerate(records):
        newRecord = [None] * len(columnNames)
        newRecord[0] = record[0]
        newRecord[1] = record[1]
        newRecord[2] = record[2]
        newRecord[3] = record[3]
        newRecord[4] = record[4]
        newRecord[5] = '()V'
        newRecord[6] = record[5]
        newRecord[7] = record[6]
        if i == 0:
            newRecord[13] = 2
        #df1 = pd.DataFrame(newRecord, columns=columnNames)
        #df.append(df1)
        df.loc[len(df)] = newRecord
        
def isRepeatedGroup(pair, pairSet):
    pair1 = pair[0]
    pair2 = pair[1]
    method1 = pair1[2] + '.' + pair1[3] + '#' + pair1[4] + '@' + str(pair1[5])
    method2 = pair2[2] + '.' + pair2[3] + '#' + pair2[4] + '@' + str(pair2[5])
    if (method1 in pairSet or method2 in pairSet):
        return True
    else:
        pairSet.add(method1)
        pairSet.add(method2)
        return False

In [2]:
fin = open(fileName, 'r')
records = fin.readlines()
fin.close()

groupID = 1
group = []
count = 0
pairSet = set()
repetition = 0
for record in records:
    tokens = record.split()
    if len(tokens) > 0:
        path = tokens[3]
        folder, package, className = getCUInfo(path)
        [startLine, length] = tokens[4].split(':')[1:3]
        kind = int(tokens[5].split(':')[1])
        startLine = int(startLine)
        length = int(length)
        method = getMethodName(path, startLine, kind)
        if method != None:
            group.append([groupID, folder, package, className, method, startLine, startLine + length - 1])
            #print([groupID, folder, package, className, method, startLine, startLine + length - 1])
    else:
        groupID += 1    
        if len(group) == 2 and checkPair(group):
            if isRepeatedGroup(group, pairSet):
                repetition += 1
            else:
                count += 1
                addRecords(group)
                #print(group)
                #print('\n')
        group = []

print(str(count) + ' pairs of refactorable candidates')
print(str(repetition) + ' repeated pairs of refactorable candidates')

144 pairs of refactorable candidates
59 repeated pairs of refactorable candidates


In [3]:
writer = pd.ExcelWriter(fout)
df.to_excel(excel_writer = writer, sheet_name = 'Sheet1', index = False)
writer.save()